In [238]:
import pandas as pd
import numpy as np
import warnings

from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [239]:
data = pd.read_csv(path+'data/ml_input.csv')
merged = pd.read_csv(path+'data/merged.csv')
parameters = pd.read_csv(path+'parameters/neutralnetworkregressor.csv')

In [240]:
N = 2022

In [241]:
params_to_drop = ['season', 'round', 'podium', 'driver_points_from']

In [242]:
df = data.copy()

train = df[df.season < N]

scaler = StandardScaler()

X_train = train.drop(params_to_drop, axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

y_train = np.asarray(train.driver_points_from.values)

In [243]:
print(*data.columns)

season round podium q_delta starting_grid driver_points_before constructor_points_before driver_points_from driver_points_per points_percentage driver_last_3 constructor_last_3 driver_adrian_sutil driver_alexander_albon driver_alexander_rossi driver_andre_lotterer driver_antonio_giovinazzi driver_brendon_hartley driver_carlos_sainz driver_charles_leclerc driver_daniel_ricciardo driver_daniil_kvyat driver_esteban_gutierrez driver_esteban_ocon driver_felipe_massa driver_felipe_nasr driver_fernando_alonso driver_george_russell driver_jack_aitken driver_jean-eric_vergne driver_jenson_button driver_jolyon_palmer driver_jules_bianchi driver_kamui_kobayashi driver_kevin_magnussen driver_kimi_raikkonen driver_lance_stroll driver_lando_norris driver_lewis_hamilton driver_marcus_ericsson driver_max_chilton driver_max_verstappen driver_mick_schumacher driver_nicholas_latifi driver_nico_hulkenberg driver_nico_rosberg driver_nikita_mazepin driver_pascal_wehrlein driver_pastor_maldonado driver_pierr

In [244]:
def get_predictions(X_train, y_train, model):
    points = []
    winners = []
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(params_to_drop, axis=1)
        y_test = test.driver_points_from

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict(X_test), columns=['predicted_points'])
        prediction_df['actual_points'] = y_test.reset_index(drop=True)

        prediction_df['predicted_winner'] = prediction_df.predicted_points.map(lambda x: 1 if x == prediction_df.predicted_points.max() else 0)
        prediction_df['actual_winner'] = prediction_df.actual_points.map(lambda x: 1 if x == prediction_df.actual_points.max() else 0)

        winners += list(prediction_df.predicted_winner.values)
        points += list(prediction_df.predicted_points.values)
        
        # Retrain model
        # X_train = pd.concat([X_train, X_test])
        # y_train = np.append(y_train, y_test)


        # model.fit(X_train, y_train)

    return points, winners

In [245]:
params = parameters.iloc[0]
hidden_layer_sizes = eval(params.hidden_layer_sizes)
hidden_layer_sizes = (50,100,50)
activation = params.activation
solver = params.solver
alpha = params.alpha

params

Unnamed: 0                        170
model                    nn_regressor
hidden_layer_sizes    (80, 20, 40, 5)
activation                   logistic
solver                           adam
alpha                        0.042813
score                        0.681818
Name: 0, dtype: object

In [246]:
test = df[(df.season == N)]

model = MLPRegressor(
    hidden_layer_sizes=hidden_layer_sizes,
    activation=activation, 
    solver=solver, 
    alpha=alpha, 
    random_state=1)
    
model.fit(X_train, y_train)

points, winner = get_predictions(X_train, y_train, model)

test['predicted_points'] = points
test['predicted_winner'] = winner

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [247]:
len(test.query('predicted_winner == 1 & podium == 1'))/ test['round'].nunique()

0.0

In [248]:
df = test[['season', 'round', 'podium', 'predicted_winner','starting_grid','driver_points_from', 'predicted_points']]

In [249]:
df.query('predicted_winner == 1').sort_values('predicted_points', ascending=False)

,season,round,podium,predicted_winner,starting_grid,driver_points_from,predicted_points
3113,2022,1,3,1,5,15.0,20.179955
3173,2022,5,6,1,6,8.0,20.175783
3249,2022,9,3,1,4,15.0,20.174321
3269,2022,10,3,1,5,16.0,20.170957
3394,2022,17,9,1,3,2.0,20.170616
3191,2022,6,5,1,6,10.0,20.170116
3152,2022,3,4,1,5,12.0,20.170110
3426,2022,19,2,1,3,18.0,20.169325
3446,2022,20,2,1,3,18.0,20.168462
3350,2022,15,4,1,4,12.0,20.164441
